In [3]:
import numpy as np

# Constants
RHO_0 = 1.225  # Sea level density (kg/m³)
G = 9.81       # Gravitational acceleration (m/s²)
KNOTS_TO_MS = 0.514444  # knots to m/s
FT_PER_NM = 6076.12     # feet per nautical mile

def get_density_ratio(altitude_ft):
    """
    Calculate density ratio ρ₀/ρ based on altitude using ISA standard atmosphere
    
    Parameters:
    - altitude_ft: Altitude in feet
    
    Returns:
    - density_ratio: ρ₀/ρ (dimensionless)
    """
    # Convert feet to meters
    altitude_m = altitude_ft * 0.3048
    
    # ISA standard atmosphere model (simplified)
    # For troposphere (0-11000m): T = T₀ - L*h, ρ/ρ₀ = (T/T₀)^(g*M/(R*L)-1)
    if altitude_m <= 11000:  # Troposphere
        T0 = 288.15  # K
        L = 0.0065   # K/m lapse rate
        T = T0 - L * altitude_m
        density_ratio = (T/T0) ** (G*0.0289644/(8.314*L) - 1)  # ρ/ρ₀
        return 1.0 / density_ratio  # ρ₀/ρ
    else:  # Simplified for higher altitudes
        # Approximate values from document
        if altitude_ft <= 10000:
            return 1.31
        elif altitude_ft <= 20000:
            return 1.75
        else:
            return 2.0  # Rough approximation

def get_descent_gradient(IAS_knots, weight_kg, S_m2, CD0, k, T_idle_N, altitude_ft=0):
    """
    Calculate descent gradient for constant IAS descent using the updated formula
    
    Parameters:
    - IAS_knots: Indicated airspeed in knots
    - weight_kg: Aircraft weight in kg
    - S_m2: Wing area in m²
    - CD0: Zero-lift drag coefficient (dimensionless)
    - k: Induced drag factor (dimensionless)
    - T_idle_N: Idle thrust in Newtons
    - altitude_ft: Current altitude in feet (default: 0)
    
    Returns:
    - sin_gamma: Descent gradient (dimensionless)
    - gamma_degrees: Flight path angle in degrees
    - gradient_ft_per_nm: Descent gradient in feet per nautical mile
    - density_ratio: ρ₀/ρ used in calculation
    """
    
    # Convert units
    IAS_ms = IAS_knots * KNOTS_TO_MS  # knots to m/s
    W_N = weight_kg * G  # kg to Newtons
    
    # Get density ratio for current altitude
    density_ratio = get_density_ratio(altitude_ft)  # ρ₀/ρ
    
    # Dynamic pressure (constant for constant IAS at sea level reference)
    q = 0.5 * RHO_0 * IAS_ms**2
    
    # Total drag calculation
    D_zero_lift = q * S_m2 * CD0  # Zero-lift drag
    D_induced = (k * W_N**2) / (q * S_m2)  # Induced drag
    D_total = D_zero_lift + D_induced
    
    # Numerator: (D - T_idle) / W
    numerator = (D_total - T_idle_N) / W_N
    
    # Denominator: 1 + (6.125×10⁻⁵×IAS²/g) × (ρ₀/ρ)
    # Note: IAS should be in m/s for this calculation
    correction_factor = (6.125e-5 * IAS_ms**2 / G) * density_ratio
    denominator = 1 + correction_factor
    
    # Final sin(γ) calculation
    sin_gamma = numerator / denominator
    
    # Convert to degrees
    gamma_rad = np.arcsin(np.clip(sin_gamma, -1, 1))  # Clip to valid range
    gamma_degrees = np.degrees(gamma_rad)
    
    # Calculate descent gradient in feet per nautical mile
    gradient_ft_per_nm = sin_gamma * FT_PER_NM
    
    return sin_gamma, gamma_degrees, gradient_ft_per_nm, density_ratio

def calculate_descent_at_multiple_altitudes(IAS_knots, weight_kg, S_m2, CD0, k, T_idle_N, 
                                          altitudes_ft):
    """
    Calculate descent gradient at multiple altitudes to show variation
    
    Parameters:
    - Same as get_descent_gradient, but altitudes_ft is a list
    
    Returns:
    - results: List of tuples (altitude, sin_gamma, gamma_deg, gradient_ft_nm, density_ratio)
    """
    results = []
    
    for alt in altitudes_ft:
        sin_gamma, gamma_deg, gradient_ft_nm, density_ratio = get_descent_gradient(
            IAS_knots, weight_kg, S_m2, CD0, k, T_idle_N, alt
        )
        results.append((alt, sin_gamma, gamma_deg, gradient_ft_nm, density_ratio))
    
    return results

if __name__ == "__main__":
    # Test parameters (A320 example)
    IAS = 280        # knots
    weight = 58000   # kg
    S = 122.6        # m²
    CD0 = 0.028      # dimensionless
    k = 0.04         # dimensionless
    T_idle = 7000    # N
    
    print("=== Updated Formula Results ===")
    print("Aircraft: A320-type")
    print(f"IAS: {IAS} knots, Weight: {weight} kg")
    print(f"CD0: {CD0}, k: {k}")
    print("-" * 50)
    
    # Single altitude calculation
    sin_gamma, gamma_deg, gradient_ft_nm, density_ratio = get_descent_gradient(
        IAS, weight, S, CD0, k, T_idle, altitude_ft=10000
    )
    
    print(f"At 10,000 ft:")
    print(f"ρ₀/ρ = {density_ratio:.3f}")
    print(f"sin(γ) = {sin_gamma:.6f}")
    print(f"γ = {gamma_deg:.3f}°")
    print(f"Descent gradient = {gradient_ft_nm:.0f} ft/nm")
    print()
    
    # Multiple altitudes to show variation
    altitudes = [0, 5000, 10000, 15000, 20000, 25000, 30000]
    results = calculate_descent_at_multiple_altitudes(
        IAS, weight, S, CD0, k, T_idle, altitudes
    )
    
    print("=== Altitude Variation Analysis ===")
    print("Alt(ft)  ρ₀/ρ    sin(γ)    γ(°)   Gradient(ft/nm)")
    print("-" * 55)
    
    for alt, sin_g, gamma_d, grad_ft_nm, rho_ratio in results:
        print(f"{alt:6d}  {rho_ratio:5.2f}  {sin_g:8.6f}  {gamma_d:5.2f}  {grad_ft_nm:8.0f}")
    
    # Show the effect of the correction factor
    print("\n=== Correction Factor Analysis ===")
    print("This shows how the denominator correction factor varies with altitude:")
    print("Alt(ft)  ρ₀/ρ   Correction Factor   Effect on sin(γ)")
    print("-" * 55)
    
    IAS_ms = IAS * KNOTS_TO_MS
    base_correction = 6.125e-5 * IAS_ms**2 / G
    
    for alt, _, _, _, rho_ratio in results[:5]:  # Show first 5 altitudes
        correction_factor = base_correction * rho_ratio
        total_factor = 1 + correction_factor
        effect_percent = (1/total_factor - 1) * 100
        print(f"{alt:6d}  {rho_ratio:5.2f}  {correction_factor:13.6f}   {effect_percent:6.2f}%")

=== Updated Formula Results ===
Aircraft: A320-type
IAS: 280 knots, Weight: 58000 kg
CD0: 0.028, k: 0.04
--------------------------------------------------
At 10,000 ft:
ρ₀/ρ = 1.354
sin(γ) = 0.067190
γ = 3.853°
Descent gradient = 408 ft/nm

=== Altitude Variation Analysis ===
Alt(ft)  ρ₀/ρ    sin(γ)    γ(°)   Gradient(ft/nm)
-------------------------------------------------------
     0   1.00  0.069921   4.01       425
  5000   1.16  0.068656   3.94       417
 10000   1.35  0.067190   3.85       408
 15000   1.59  0.065492   3.76       398
 20000   1.88  0.063528   3.64       386
 25000   2.23  0.061262   3.51       372
 30000   2.67  0.058658   3.36       356

=== Correction Factor Analysis ===
This shows how the denominator correction factor varies with altitude:
Alt(ft)  ρ₀/ρ   Correction Factor   Effect on sin(γ)
-------------------------------------------------------
     0   1.00       0.129548   -11.47%
  5000   1.16       0.150355   -13.07%
 10000   1.35       0.175450   -14.